### Ingestion de la carpeta "movie_company"

In [0]:
dbutils.widgets.text('p_enviroment', '')
v_enviroment = dbutils.widgets.get('p_enviroment')

In [0]:
dbutils.widgets.text('p_file_date', '2024-12-16')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo CSV usando DataFrameReader de Spark

In [0]:
from pyspark.sql.types import *

In [0]:
movies_companies_schema = StructType(fields=[
    StructField('movieId', IntegerType(), True),
    StructField('companyId', IntegerType(), True)
])

In [0]:
movies_companies_df = spark.read \
    .schema(movies_companies_schema) \
    .csv(f'{bronze_folder_path}/{v_file_date}/movie_company')

### Paso 2 - Renombrar y Agregar Columnas
1. "movieId" renombrar a "movie_id"
2. "companyId" renombrar a "company_id"
3. Agregar columna "ingestion_date"
4. Agregar columna "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_companies_final_df = add_ingestion_date(movies_companies_df) \
    .withColumnRenamed('movieId', 'movie_id') \
    .withColumnRenamed('companyId', 'company_id') \
    .withColumn('enviroment', lit(v_enviroment)) \
    .withColumn('file_date', lit(v_file_date))

### Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movies_companies_final_df, 'movie_silver', 'movies_companies', 'file_date')

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(movies_companies_final_df, "movie_silver", "movies_companies", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movies_companies
GROUP BY file_date;

In [0]:
display(spark.read.format('delta').load(f'{silver_folder_path}/movies_companies'))

In [0]:
dbutils.notebook.exit("Success")